### Setup

In [ ]:
%%sh

export PIP_ROOT_USER_ACTION=ignore

pip install -Uq pip
pip install autotrain-advanced==0.6.58
pip install diffusers==0.21.4
pip install autocrop

### > check version

In [ ]:
import torch

print(torch.__version__) # e.g., 2.0.0 at time of post

print(torch.cuda.get_device_name(0)) # e.g., NVIDIA A10G

device_count = torch.cuda.device_count()
assert device_count > 0, "No GPU devices detected."

print("Number of available GPU devices:", device_count)

device = torch.device("cuda")

### > Prepare the images. The picture needs to be 1024 x 1024

In [ ]:
from tqdm.notebook import tqdm
import os
from pathlib import Path
from itertools import chain
import utils
import shutil

imag_dir=Path("data")
dest_dir = Path("cropped")
dest_dir.mkdir(parents=True, exist_ok=True)

for n,img_path in enumerate(chain(imag_dir.glob("*.[jJ][pP]*[Gg]"),imag_dir.glob("*.[Pp][Nn][Gg]"))):
    try:
        cropped = utils.resize_and_center_crop(img_path.as_posix(), 1024)
        cropped.save(dest_dir / f"image_{n}.png")
    except ValueError:
        print(f"Could not detect face in {img_path}. Skipping.")
        continue

print("Here are the preprocessed images ==========")
[x.as_posix() for x in dest_dir.iterdir() if x.is_file()]

- 8bit adam gobbles the images
- prior-preservation exceeds A10G GPU memory
- xformers gives package error

### > Initialize fine tuning parameters

In [1]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
# project configuration
project_name = "finetune_jingswu"

model_name_base = "stabilityai/stable-diffusion-xl-base-1.0"
model_name_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"
model_name_upscaler_4x = "stabilityai/stable-diffusion-x4-upscaler"
model_name_latent_upscaler = "stabilityai/sd-x2-latent-upscaler"

# fine-tuning prompts
# 'oue' is the rare tokens, 'car' is the class
instance_prompt = "photo of <<TOK>>"
class_prompt = "photo of a person"

# fine-tuning hyperparameters
learning_rate = 1e-4
num_steps = 1000
batch_size = 1
gradient_accumulation = 4
resolution = 1024
num_class_image = 50

class_image_path=Path(f"/tmp/priors")

# environment variables for autotrain command
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name_base
os.environ["INSTANCE_PROMPT"] = instance_prompt
os.environ["CLASS_PROMPT"] = class_prompt
os.environ["IMAGE_PATH"] = dest_dir.as_posix()
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["CLASS_IMAGE_PATH"] = class_image_path.as_posix()
os.environ["NUM_CLASS_IMAGE"] = str(num_class_image)

### > use autotrain to fine tune

help command will show all the available parameters

```
!autotrain dreambooth --help
```

In [ ]:
!autotrain dreambooth \
    --model ${MODEL_NAME} \
    --project-name ${PROJECT_NAME} \
    --image-path "${IMAGE_PATH}" \
    --prompt "${INSTANCE_PROMPT}" \
    --class-prompt "${CLASS_PROMPT}" \
    --resolution ${RESOLUTION} \
    --batch-size ${BATCH_SIZE} \
    --num-steps ${NUM_STEPS} \
    --gradient-accumulation ${GRADIENT_ACCUMULATION} \
    --lr ${LEARNING_RATE} \
    --fp16 \
    --gradient-checkpointing

### > Load the fine tune model

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline

pipeline = DiffusionPipeline.from_pretrained(
    model_name_base,
    torch_dtype=torch.float16,
).to(device)

pipeline.load_lora_weights(
    project_name, 
    weight_name="pytorch_lora_weights.safetensors",
    adapter_name="jingswu"
)

In [ ]:
subject_prompt = """photo of <<TOK>>, epic front portrait, with glasses, zoom in, suit and tie, young and handsome, symmetry, blured background, futuristic cityscape"""

subject_negative_prompt = """ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, 
watermark, grainy, signature, cut off, draft, amateur, multiple, gross, weird, uneven, furnishing, decorating, decoration, furniture, text, poor, low, basic, worst, juvenile, 
unprofessional, failure, crayon, oil, label, thousand hands"""

In [ ]:
import random

seed = random.randint(0, 100000)
generator = torch.Generator(device).manual_seed(seed)
base_image = pipeline(
    prompt=subject_prompt, 
    negative_prompt=subject_negative_prompt,
    num_inference_steps=50,
    generator=generator,
    height=1024,
    width=1024,
    output_type="pil",
).images[0]
base_image

## Load the weights file into model package



In [ ]:
import sagemaker
import boto3
import time
import json
from io import BytesIO
import base64
from PIL import Image

from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

s3_prefix = (
    "stable-diffusion-dreambooth-workshop"  # folder within bucket where code artifact will go
)

region = sess._region_name
account_id = sess.account_id()
mme_prefix = f"{s3_prefix}/inference/models"

model_data_url = f"s3://{bucket}/{mme_prefix}/"

In [ ]:
model_repo = "model_dir"

model_name = "jingswu"
lora_weights = f"{project_name}/pytorch_lora_weights.safetensors"

dest_dir = f"{model_repo}/{model_name}/1"

shutil.copy(lora_weights, dest_dir)

Create the tar file

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
tar_name = f"{model_name}.tar.gz"
!tar -C $model_repo -czvf $tar_name $model_name

sess.upload_data(path=tar_name, bucket=bucket, key_prefix=mme_prefix)
!rm $tar_name

## Deploy endpoint
Now, you get the customer container url

In [ ]:
%store -r extended_triton_image_uri

you are now ready to configure and deploy the multi-model endpoint

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

container = {
    "Image": extended_triton_image_uri,
    "ModelDataUrl": model_data_url, #f"{model_data_url}{tar_name}",     # S3 location of the models
#     "Environment": {"SAGEMAKER_TRITON_DEFAULT_MODEL_NAME": "jingswu"}
    "Mode": "MultiModel",
}

In [ ]:
sm_model_name = name_from_base(f"{mme_prefix.split('/')[0]}-models")

create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

Create a SageMaker endpoint configuration.



In [ ]:
endpoint_config_name = name_from_base(f"{mme_prefix.split('/')[0]}-epc")

instance_type = 'ml.g5.2xlarge'

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": instance_type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

Create the endpoint, and wait for it to transition to InService state. (This takes about 5 mins)

In [ ]:
endpoint_name = name_from_base(f"{mme_prefix.split('/')[0]}-ep")

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## Invoke the LoRA fine tuned model

In [ ]:
sm_runtime = boto3.client("sagemaker-runtime")

In [ ]:
# helper functions to encode and decode images
def decode_image(img):
    buff = BytesIO(base64.b64decode(img.encode("utf8")))
    image = Image.open(buff)
    return image

In [ ]:
import random

prompt = """photo of <<TOK>>, epic front portrait, with glasses, zoom in, suit and tie, young and handsome, symmetry, blured background, futuristic cityscape"""

negative_prompt = """ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, 
watermark, grainy, signature, cut off, draft, amateur, multiple, gross, weird, uneven, furnishing, decorating, decoration, furniture, text, poor, low, basic, worst, juvenile, 
unprofessional, failure, crayon, oil, label, thousand hands"""

seed = random.randint(1, 1000000000)
gen_args = json.dumps(dict(num_inference_steps=50, guidance_scale=7, height=1024, width=1024, seed=seed))

inputs = dict(prompt = prompt,
              negative_prompt = negative_prompt,
              gen_args = gen_args)

payload = {
    "inputs":
        [{"name": name, "shape": [1,1], "datatype": "BYTES", "data": [data]} for name, data in inputs.items()]
}

In [ ]:
%%time
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/octet-stream",
    Body=json.dumps(payload),
    TargetModel=tar_name,
)
output = json.loads(response["Body"].read().decode("utf8"))["outputs"]
original_image = decode_image(output[0]["data"][0])
original_image

## Clean Up

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=sm_model_name)